# Lab 5 Submission

Author: Lucy Roberts

Date: 2/7/2022

Class: Geospatial Data Science

Prof: Johnny Ryan

GE: Insang Song

In [1]:
#MUST UPDATE ENVIRONMENT TO INCLUDE LAB 2 AND 3
# !conda env update -n lab5 -f environment_lab2.yml environment_lab3.yml

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV

import osmnx as ox 
import networkx as nx
import folium

from cenpy import products


# Import data
df = pd.read_csv("C:\\Users\\lucin\\OneDrive\\Desktop\\GeospatialDataScience\\geospatial-data-science\\labs\\lab5\\lab5data\\seattle_house_prices.csv")

In [3]:
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry import LineString, MultiLineString

In [28]:
# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat']))
gdf = gdf.set_crs(4326, allow_override=True)

# Reproject everything to UTM 10N (EPSG:32610)
gdf_utm = gdf.to_crs('EPSG:32610')

### Question 1

1. There are _19451_ houses in the dataset.

2. There are 7 features for predicting house price.

3. There are no null features of the dataset.

In [5]:
null_gdf = gdf.info()
null_gdf

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19451 entries, 0 to 19450
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   price        19451 non-null  int64   
 1   bedrooms     19451 non-null  int64   
 2   bathrooms    19451 non-null  float64 
 3   sqft_living  19451 non-null  int64   
 4   sqft_lot     19451 non-null  int64   
 5   yr_built     19451 non-null  int64   
 6   lat          19451 non-null  float64 
 7   long         19451 non-null  float64 
 8   geometry     19451 non-null  geometry
dtypes: float64(3), geometry(1), int64(5)
memory usage: 1.3 MB


4. sqrt_living (correl = 0.702), bathrooms (correl = 0.524), and bedrooms (correl = 0.315) are the most correlated with housing price.

5. sqft_lot (correl = 0.090), yr_built (correl = 0.052), and long (correl = 0.020) are least correlated with housing price.

In [26]:
# Compute correlation matrix
corr_matrix = gdf_utm.corr()

# Display just house value correlations
corr_matrix["price"].sort_values(ascending= False)

price             1.000000
sqft_living       0.702296
BachPlus_ByPop    0.531980
bathrooms         0.524395
B25119_001E       0.509853
B15003_024E       0.448381
B15003_025E       0.406826
B25064_001E       0.393263
B15003_023E       0.365436
bedrooms          0.315804
lat               0.308082
B15003_022E       0.272551
White_byPop       0.210071
B03002_003E       0.139224
B25091_001E       0.112195
sqft_lot          0.090125
index_right       0.053405
yr_built          0.052453
long              0.020092
B01003_001E       0.016883
B25092_001E      -0.258037
Name: price, dtype: float64

## Question 2

### Add extra features

####

In [7]:
# Define product
acs = products.ACS(2019)

###### Variables
'B01003' = total population
B25091_001 = number of homeowners in residence

B15003_025E Dr
B15003_024E Professional
B15003_023E Master's
B15003_022E Bachelor's

B25119_001E Median hh income

B25092_001E monthly owner costs

###### Calculated 
BachPlus_ByPop- sum of all advanced degrees divided by total pop

White_ByPop- number of white people by total pop

In [27]:
#Download Data
king_cost = products.ACS(2019).from_county('King County, WA', level='tract',
                                        variables=['B25091_001E', 'B15003_025E', 'B15003_024E', 'B15003_023E', 'B15003_022E', 'B01003_001E', 'B25119_001E', 'B03002_003E', 'B25064_001E'])

# Calculate some stats
king_cost['BachPlus_ByPop'] = (king_cost['B15003_025E'] + king_cost['B15003_024E'] + king_cost['B15003_023E'] + king_cost['B15003_022E'])/king_cost['B01003_001E']
king_cost['White_byPop'] = king_cost['B03002_003E'] / king_cost['B01003_001E']

C:\Users\lucin\anaconda3\envs\lab5\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lucin\anaconda3\envs\lab5\lib\site-packages\pandas\core\dtypes\cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
C:\Users\lucin\anaconda3\envs\lab5\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.gith

In [29]:
#Make df
king_gdf = gpd.GeoDataFrame(king_cost, geometry='geometry')
king_gdf = king_gdf.to_crs('EPSG:32610')

In [30]:
joined = gdf_utm.sjoin(king_gdf, how="inner", predicate='intersects')

In [31]:
gdf_utm = joined

##### Proximity to university

In [12]:
# Specify type of data
tags = {'amenity': 'university'}

# Download building geometries from OSM
colleges = ox.geometries_from_place('Seattle, WA, USA', tags)


colleges = colleges.to_crs('EPSG:32610')

# Get bathroom and EMU centroids
colleges['centroid'] = colleges['geometry'].apply(
  lambda x: x.centroid if type(x) == Polygon else (
  x.centroid if type(x) == MultiPolygon else x))


# Convert back to WGS84
colleges['centroid']= colleges['centroid'].to_crs('EPSG:4326')

C:\Users\lucin\anaconda3\envs\lab5\lib\site-packages\osmnx\geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [13]:
#Calculate distance
colleges_utm = colleges['centroid'].to_crs('EPSG:32610')

distance_to_uni = []
VeryClose = []
for i in range(gdf_utm.shape[0]):
    dist = colleges_utm.distance(gdf_utm['geometry'].iloc[i]).min()
    distance_to_uni.append(dist)
    if dist < 5000:
        VeryClose.append(1)
    else:
        VeryClose.append(0)
        
#walk to uni is actually just close to uni, within 5 km        
gdf_utm['walk_to_uni'] = VeryClose

##### Distance to Coast

In [14]:
# Read dataset
coast = gpd.read_file("C:\\Users\\lucin\\OneDrive\\Desktop\\GeospatialDataScience\\geospatial-data-science\\labs\\lab5\\lab5data\\washington_coastline.shp")

#Convert to UTM 10N
coast_utm = coast.to_crs('EPSG:32610')

# Compute distance to coast
distance_to_coast = []
OnWater = []
for i in range(gdf_utm.shape[0]):
    dist = coast_utm.distance(gdf_utm['geometry'].iloc[i]).min()
    distance_to_coast.append(dist)
    if dist < 500:
        OnWater.append(1)
    else:
        OnWater.append(0)

In [15]:
gdf_utm['OnWater'] = OnWater #Should select only the data that are very close to the shore.

#### Create model

In [17]:
# Define feature list
feature_list =  ['sqft_living', 'bathrooms', 'bedrooms', 'sqft_lot', 'lat', 'BachPlus_ByPop', 'B25119_001E', 'White_byPop', 'walk_to_uni', 'OnWater']

# Define features and labels 
X = gdf_utm[feature_list]
y = gdf_utm['price']

# Standarize data
scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)

In [18]:
# Split data 
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [19]:
# Define model
forest_reg = RandomForestRegressor(n_estimators=830, min_samples_leaf=1, max_features='sqrt', bootstrap = True, n_jobs = -1, oob_score=True)

# Fit model
forest_reg.fit(X_train, y_train)

RandomForestRegressor(max_features='sqrt', n_estimators=830, n_jobs=-1,
                      oob_score=True)

In [20]:
# Predict test labels predictions
predictions = forest_reg.predict(X_test)

# Compute mean-squared-error
final_mse = metrics.mean_squared_error(y_test , predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

146648.6060922433

In [37]:
#hyper parameter testing from https://towardsdatascience.com/predicting-housing-prices-using-a-scikit-learns-random-forest-model-e736b59d56c5
# "Predicting Housing Prices Using Scikit-Learn's Random Forest Model" by Santosh Yadaw
"""
n_estimators = [int(x) for x in np.arange(start = 10, stop = 2000, step = 10)]
max_features = [0.5,'auto', 'sqrt','log2']
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
# First create the base model to tune
m = RandomForestRegressor()
# Fit the random search model
m_random = RandomizedSearchCV(estimator = m, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
m_random.fit(X_train, y_train)
m_random.best_params_
"""

Fitting 3 folds for each of 100 candidates, totalling 300 fits


{'n_estimators': 830,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'bootstrap': False}